In [1]:
# ⚙️ Cài đặt YOLOv8 + DeepSORT
!pip install -q ultralytics deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.5 MB/s eta 0:00:00


In [5]:
# Mount Google Drive nếu cần
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Lấy đường dẫn file
video_path = "/content/drive/MyDrive/dispatch/1473_CH05_20250501133703_154216.mp4"
print(f"📂 Video đã upload: {video_path}")

📂 Video đã upload: /content/drive/MyDrive/dispatch/1473_CH05_20250501133703_154216.mp4


In [2]:
# 🚀 Tracking YOLOv8 + DeepSORT
from pathlib import Path
import cv2, pickle
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

model = YOLO("/content/drive/MyDrive/dispatch/best.pt")
tracker = DeepSort(max_age=30)

output_log_path = Path("bbox_tracking_log.pkl")
bbox_log = {}

cap = cv2.VideoCapture(video_path)
frame_idx = 0

while True:
    success, frame = cap.read()
    if not success:
        break

    results = model.predict(source=frame, imgsz=640, conf=0.1, verbose=False)[0]

    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cls_id = int(box.cls[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], 0.9, cls_id))

    if not detections:
        frame_idx += 1
        continue

    tracks = tracker.update_tracks(detections, frame=frame)
    bbox_log[frame_idx] = []

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cls_name = model.names[track.det_class] if hasattr(track, "det_class") else "object"

        bbox_log[frame_idx].append({
            "id": int(track_id),
            "object": cls_name,
            "bbox": [x1, y1, x2 - x1, y2 - y1]
        })

    frame_idx += 1

cap.release()

with open(output_log_path, "wb") as f:
    pickle.dump(bbox_log, f)

print("✅ Tracking hoàn tất. Log lưu tại:", output_log_path)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Tracking hoàn tất. Log lưu tại: bbox_tracking_log.pkl


In [4]:
# 📤 Tải file log về máy
from google.colab import files
files.download("bbox_tracking_log.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>